In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from qids_package.qids import *

In [2]:
env = make_env()  # initialize the environment

Environment is initialized.


In [3]:
# sample code using random number as prediction
import random
random.seed(42)
while not env.is_end():
    fundamental_df, market_df = env.get_current_market()  # get correlated dataset

    # make your prediction Y here and replace the following four rows
    l = []
    for idx in range(54):
        l.append(random.random())
    predict_ds = pd.Series(l)

    env.input_prediction(predict_ds)  # upload your predicted Y

Data Feeding is finished.


In [4]:
import pandas as pd
print(pd.read_csv('./submission.csv'))

      date_time    return
0       s0d1001  0.639427
1       s1d1001  0.025011
2       s2d1001  0.275029
3       s3d1001  0.223211
4       s4d1001  0.736471
...         ...       ...
37795  s49d1700  0.073287
37796  s50d1700  0.079234
37797  s51d1700  0.920565
37798  s52d1700  0.261712
37799  s53d1700  0.054814

[37800 rows x 2 columns]
